Action1
使用MinHashLSHForest对微博新闻句子进行检索 weibo.txt
针对某句话进行Query，查找Top-3相似的句子
1、完成代码（30points）
2、使用MinHashLSHForest工具对MinHash进行index，并完成Query Top-K，代码正确（30points）

In [8]:
pip install datasketch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
# 对微博进行相似句子Top-K查询
from datasketch import MinHash, MinHashLSH, MinHashLSHForest
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re
# 读取文件
f = open('./weibos.txt', 'r', encoding='UTF-8')
text = f.read()
print(text)

#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用。 ​
国足输给叙利亚之后，里皮辞职。谁将成为新主帅，成为广大球迷关注的焦点。目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里。 ​
据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决。据了解。比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战。辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利。这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话。至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商。
让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……。来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行
斯科拉里想执教中国国足！老头有点意思，凡是里皮干过的地方，他就想试试。当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛！可以试试！
斯科拉里的水平，还不如里皮。斯科拉里，看好的不是国足，而是年薪…… ​
非常应该辞职！中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有！从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了！穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪
赛后，叙利亚主教练在更衣室里给每个队员一个耳光。主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队！中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去！”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛？”
里皮辞职返回意大利，他的助教马达洛尼随队返回广州。马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事。”
中国足协：接受里皮辞职请求，将深刻反思
看了个报道，马达洛尼说：“关于里皮的辞职，我事先也没有被告知，自己也不清楚发生了什么，也许是里皮头脑一热的决定。足协也可

In [7]:
# 以句号，叹号，问号作为分隔，去掉\n换行符号
sentences = re.split('[。！？]', text.replace('\n', ''))

# 最后一行如果为空，则删除
if sentences[len(sentences)-1] == '':
    sentences.pop()
print(sentences)
print(len(sentences))

['#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用', ' \u200b国足输给叙利亚之后，里皮辞职', '谁将成为新主帅，成为广大球迷关注的焦点', '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里', ' \u200b据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决', '据了解', '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战', '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利', '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话', '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商', '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……', '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足', '老头有点意思，凡是里皮干过的地方，他就想试试', '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛', '可以试试', '斯科拉里的水平，还不如里皮', '斯科拉里，看好的不是国足，而是年薪…… \u200b非常应该辞职', '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有', '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了', '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光', '主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队', '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去', '”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛', '”里皮辞职返回意大利，他的助教马达洛尼随队返回广州', '马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事',

In [8]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str

In [9]:
# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [11]:
# 设置停用词
stop = [line.strip().decode('utf-8') for line in open('stopword.txt').readlines()]
# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

In [13]:
# 创建LSH Forest及MinHash对象
minhash_list = []
# 创建LSH Forest
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()

In [20]:
query = '斯科拉里愿意执教国足'
# 将item_text进行分词
item_str = get_item_str(query)
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

# 查询forest中与minhash_query相似的Top-K（TOP-3）个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i],'  ', minhash_query.jaccard(minhash_list[result[i]]),'  ', documents[result[i]].replace(' ', ''))
print("Top 3 邻居:", result)

0    0.3046875    #斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用
10    0.1875    让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……
13    0.1484375    当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛
Top 3 邻居: [0, 10, 13]
